In [6]:
from comet_ml import API
import pandas as pd
import os
import sys
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

current_dir = os.path.abspath(os.getcwd())
parent_dir = os.path.dirname(current_dir)
sys.path.append(os.path.join(parent_dir, 'scripts'))

from Plots import Centiles_plot, ROC_plot, cumulative_centiles_plot, calibrate_display
from feature_engineering import preprocessing

test_data1 = pd.read_csv('../data/derivatives/test_data_saison_eli.csv')
test_data2 = pd.read_csv('../data/derivatives/test_data_saison_reg.csv')

In [4]:
API_KEY = os.environ.get('COMET_API_KEY')
project_name = "milestone-2"
workspace = "me-pic"

try:
    api = API(api_key=API_KEY)
    experiment = api.get("me-pic/milestone-2/e9e4c038bb1e44b8ac47a342db6ecb40")
    print(experiment)
    experiment.download_model("HistGradientBoosting", output_path="../model", expand=True)
    api.download_registry_model(project_name, "HistGradientBoosting", "1.0.0", output_path="../model", expand=True)
except Exception as e:
    print(f"An error occurred: {e}")
    
with open('../model/histGBmodel.pickle', 'rb') as file:
    model = pickle.load(file)
    
X, y = preprocessing(test_data1, 'is_goal')
    
predictions = model.predict(test_data1)
    
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

COMET INFO: Downloading experiment model 'HistGradientBoosting' ...


<APIExperiment 'me-pic/milestone-2/e9e4c038bb1e44b8ac47a342db6ecb40'>


COMET INFO: Unzipping model to '../model' ...
COMET INFO: done!
COMET INFO: Downloading registry model 'HistGradientBoosting', version '1.0.0', stage None from workspace 'milestone-2'...


An error occurred: GET https://www.comet.com/api/rest/v2/registry-model/item/download?workspaceName=milestone-2&modelName=HistGradientBoosting&version=1.0.0 failed with status code 403: Not allowed to see model


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- away_team_skater_count
- gameId
- home_team_skater_count
- index
- last_event_type
- ...
Feature names seen at fit time, yet now missing:
- last_event_type_BLOCKED_SHOT
- last_event_type_FACEOFF
- last_event_type_GIVEAWAY
- last_event_type_GOAL
- last_event_type_HIT
- ...
